In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

In [2]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
weather_df = pd.read_sql_query('select * from weatherinszeged',con=engine)

engine.dispose()

weather_df.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


In [3]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96453 entries, 0 to 96452
Data columns (total 12 columns):
date                   96453 non-null datetime64[ns, UTC]
summary                96453 non-null object
preciptype             96453 non-null object
temperature            96453 non-null float64
apparenttemperature    96453 non-null float64
humidity               96453 non-null float64
windspeed              96453 non-null float64
windbearing            96453 non-null float64
visibility             96453 non-null float64
loudcover              96453 non-null float64
pressure               96453 non-null float64
dailysummary           96453 non-null object
dtypes: datetime64[ns, UTC](1), float64(8), object(3)
memory usage: 8.8+ MB


In [4]:
weather_df["difference"] = weather_df['temperature'] - weather_df['apparenttemperature']
weather_df.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary,difference
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.,2.083333
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.,2.127778
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.,0.000000
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.,2.344444
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.,1.777778


In [6]:
weather_df["hum_wind"] = weather_df.humidity * weather_df.windspeed

Y = weather_df["difference"]
X = weather_df[["humidity", "windspeed"]]

X = sm.add_constant(X)

results = sm.OLS(Y,X).fit()

results.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             difference   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Thu, 06 Jun 2019   Prob (F-statistic):               0.00
Time:                        10:40:15   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.4381      0.021   -115.948      0.000      -2.479      -2.397
humidity       3.0292      0.024    126.479      0.000       2.982       3.076
windspeed      0.1193      0.001    176.164      0.000       0.118       0.121
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                           0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
Y = weather_df["difference"]
X = weather_df[["humidity", "windspeed", "hum_wind"]]

X = sm.add_constant(X)

results = sm.OLS(Y,X).fit()

results.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             difference   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Thu, 06 Jun 2019   Prob (F-statistic):               0.00
Time:                        10:40:55   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0839      0.033     -2.511      0.012      -0.149      -0.018
humidity      -0.1775      0.043     -4.133      0.000      -0.262      -0.093
windspeed     -0.0905      0.002    -36.797      0.000      -0.095      -0.086
hum_wind       0.2971      0.003     88.470      0.000       0.291       0.304
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                           0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In the first model, the r-squared score was low (.288), this means that our predictions are not reliable. This is a low r-squared and adjusted r-spuared. 

When we include the humidity and windspeed, the r-sqaured goes up to (.341) which is better, but is not very high still. This is not a very reliable set of models for what we are looking for.

In [10]:
Y = weather_df["difference"]
X = weather_df[["humidity", "windspeed", "visibility"]]

X = sm.add_constant(X)

results = sm.OLS(Y,X).fit()

results.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             difference   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                 1.401e+04
Date:                Thu, 06 Jun 2019   Prob (F-statistic):               0.00
Time:                        10:52:47   Log-Likelihood:            -1.6938e+05
No. Observations:               96453   AIC:                         3.388e+05
Df Residuals:                   96449   BIC:                         3.388e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.5756      0.028    -56.605      0.000      -1.630      -1.521
humidity       2.6066      0.025    102.784      0.000       2.557       2.656
windspeed      0.1199      0.001    179.014      0.000       0.119       0.121
visibility    -0.0540      0.001    -46.614      0.000      -0.056      -0.052
==============================================================================
Omnibus:                     3833.895   Durbin-Watson:                   0.282
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4584.022
Skew:                           0.459   Prob(JB):                         0.00
Kurtosis:                       3.545   Cond. No.                         131.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The implementation of the visibility feature improves the overall r-squared, but it is still really low (.304, .303). This model would be more useful than the first, but again this is not very reliable based on the adjusted r-squared value.

It appears that the original model has the highest AIC and BIC scores and would be the best fit out of these three choices.

In [11]:
import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

In [12]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
house_df = pd.read_sql_query('select * from houseprices',con=engine)

engine.dispose()

house_df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


In [13]:
house_df = pd.concat([house_df, pd.get_dummies(house_df.neighborhood, prefix="neighborhood", drop_first=True)], axis=1)

house_df = pd.concat([house_df, pd.get_dummies(house_df.housestyle, prefix="housestyle", drop_first=True)], axis=1)

dummy_column_names = list(pd.get_dummies(house_df.neighborhood, prefix="neighborhood", drop_first=True).columns)

dummy_column_names = list(pd.get_dummies(house_df.housestyle, prefix="housestyle", drop_first=True).columns)

In [15]:
Y2 = house_df["saleprice"]
X2 = house_df[["overallqual", "firstflrsf", "garagecars", "totalbsmtsf", "grlivarea"] + dummy_column_names]

X2 = sm.add_constant(X2)

results = sm.OLS(Y2,X2).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.771
Model:                            OLS   Adj. R-squared:                  0.769
Method:                 Least Squares   F-statistic:                     406.5
Date:                Thu, 06 Jun 2019   Prob (F-statistic):               0.00
Time:                        11:02:10   Log-Likelihood:                -17467.
No. Observations:                1460   AIC:                         3.496e+04
Df Residuals:                    1447   BIC:                         3.503e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const             -1.238e+05   5816.846    -21.290      0.000   -1.35e+05   -1.12e+05
overallqual        2.428e+04   1079.385     22.491      0.000    2.22e+04    2.64e+04
firstflrsf          -29.5523      8.409     -3.515      0.000     -46.047     -13.058
garagecars         1.646e+04   1760.096      9.354      0.000     1.3e+04    1.99e+04
totalbsmtsf          24.3748      4.290      5.681      0.000      15.959      32.791
grlivarea            79.0552      6.090     12.982      0.000      67.110      91.000
housestyle_1.5Unf  1.713e+04   1.12e+04      1.533      0.125   -4784.166     3.9e+04
housestyle_1Story   3.22e+04   5051.738      6.375      0.000    2.23e+04    4.21e+04
housestyle_2.5Fin -4.478e+04   1.49e+04     -2.998      0.003   -7.41e+04   -1.55e+04
housestyle_2.5Unf -4.348e+04   1.21e+04     -3.596      0.000   -6.72e+04   -1.98e+04
housestyle_2Story -3594.2839   4223.441     -0.851      0.395   -1.19e+04    4690.438
housestyle_SFoyer  3.379e+04   7685.032      4.397      0.000    1.87e+04    4.89e+04
housestyle_SLvl    2.276e+04   6232.400      3.652      0.000    1.05e+04     3.5e+04
==============================================================================
Omnibus:                      416.046   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            32007.362
Skew:                          -0.311   Prob(JB):                         0.00
Kurtosis:                      25.929   Cond. No.                     3.49e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.49e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
Y3 = house_df["saleprice"]
X3 = house_df[["overallqual", "firstflrsf", "garagecars", "totalbsmtsf", "grlivarea"]]

X3 = sm.add_constant(X3)

results = sm.OLS(Y3,X3).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.762
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     928.8
Date:                Thu, 06 Jun 2019   Prob (F-statistic):               0.00
Time:                        11:04:41   Log-Likelihood:                -17497.
No. Observations:                1460   AIC:                         3.501e+04
Df Residuals:                    1454   BIC:                         3.504e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        -1.03e+05   4901.609    -21.006      0.000   -1.13e+05   -9.33e+04
overallqual  2.396e+04   1083.493     22.109      0.000    2.18e+04    2.61e+04
firstflrsf     11.6608      5.024      2.321      0.020       1.805      21.516
garagecars   1.819e+04   1752.914     10.377      0.000    1.48e+04    2.16e+04
totalbsmtsf    25.0167      4.299      5.819      0.000      16.583      33.450
grlivarea      43.2993      2.678     16.170      0.000      38.047      48.552
==============================================================================
Omnibus:                      417.210   Durbin-Watson:                   1.973
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            35788.405
Skew:                          -0.254   Prob(JB):                         0.00
Kurtosis:                      27.250   Cond. No.                     1.11e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.11e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [18]:
Y4 = house_df["saleprice"]
X4 = house_df[["overallqual", "firstflrsf", "garagecars", "totalbsmtsf", "grlivarea", "lotarea"] + dummy_column_names]

X4 = sm.add_constant(X4)

results = sm.OLS(Y4,X4).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.776
Model:                            OLS   Adj. R-squared:                  0.774
Method:                 Least Squares   F-statistic:                     385.9
Date:                Thu, 06 Jun 2019   Prob (F-statistic):               0.00
Time:                        11:08:23   Log-Likelihood:                -17451.
No. Observations:                1460   AIC:                         3.493e+04
Df Residuals:                    1446   BIC:                         3.500e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              -1.27e+05   5781.778    -21.973      0.000   -1.38e+05   -1.16e+05
overallqual        2.507e+04   1076.756     23.279      0.000     2.3e+04    2.72e+04
firstflrsf          -32.1970      8.331     -3.865      0.000     -48.540     -15.854
garagecars           1.6e+04   1743.091      9.181      0.000    1.26e+04    1.94e+04
totalbsmtsf          22.5882      4.256      5.308      0.000      14.240      30.937
grlivarea            77.0074      6.035     12.760      0.000      65.169      88.846
lotarea               0.6029      0.106      5.704      0.000       0.396       0.810
housestyle_1.5Unf   1.79e+04   1.11e+04      1.620      0.105   -3777.113    3.96e+04
housestyle_1Story   3.34e+04   5001.925      6.676      0.000    2.36e+04    4.32e+04
housestyle_2.5Fin -4.236e+04   1.48e+04     -2.866      0.004   -7.14e+04   -1.34e+04
housestyle_2.5Unf -4.209e+04    1.2e+04     -3.519      0.000   -6.56e+04   -1.86e+04
housestyle_2Story -2778.8855   4180.599     -0.665      0.506    -1.1e+04    5421.801
housestyle_SFoyer  3.489e+04   7605.055      4.587      0.000       2e+04    4.98e+04
housestyle_SLvl    2.328e+04   6166.240      3.775      0.000    1.12e+04    3.54e+04
==============================================================================
Omnibus:                      446.252   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            39302.781
Skew:                          -0.405   Prob(JB):                         0.00
Kurtosis:                      28.405   Cond. No.                     2.23e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.23e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Running the original model for house prices, I find that the r-squared and adjusted r-squared values are in a good range. Meaning that the predictions from this model are reliable. 

Removing the categorical features reduces the r-squared values, but increases the AIC and BIC scores. This tells me that the second model is a better fit, but the first model has a more reliable prediction?

By adding the lot area feature, the r-spuared and adjusted r-squared both went up. The AIC and BIC scores dip a little but they are comparable. It seems like the third model I created would be the most reliable and have a good fit.